In [4]:
import os
import json
import pandas as pd
import traceback

In [8]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

In [10]:
KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
ChatOpenAI(llm=)